# C++ 测试

In [1]:
#include <cassert>
#include <iostream>
#include <sstream>
#include <vector>

typedef int32_t index_t;   // 用于标识张量索引的类型
typedef uint32_t shape_t;  // 用于标识张量形状的类型

In [2]:
// 内部函数
std::vector<shape_t> cumrod(std::vector<shape_t> shape) {
  // ----------- 累积乘法-----------
  std::vector<shape_t> factors = {};
  index_t factor = 1;
  for (auto i : shape) {
    factor *= i;
    factors.push_back(factor);
  }
  return factors;
}

In [3]:
std::vector<shape_t> shape_step(std::vector<shape_t> shape) {
  // 计算 reshape 对应维度的跨步
  std::vector<shape_t> factors{shape.rbegin(), shape.rend() - 1};  // 切片, 倒序
  std::vector<shape_t> out = cumrod(factors);
  std::vector<shape_t> factors2{out.rbegin(), out.rend()};
  factors2.push_back(1);
  return factors2;
}

In [4]:
shape_t dot_shape(std::vector<shape_t> shape, std::vector<shape_t> shape2) {
  // 形状内积
  shape_t index = 0;
  for (shape_t t = 0; t < shape.size(); t++) {
    index += shape[t] * shape2[t];
  }
  return index;
}

In [5]:
std::vector<int8_t> processing(int8_t* src_addr, uint32_t pad_top, uint32_t pad_left,
                               std::vector<shape_t> src_shape, std::vector<shape_t> src_steps,
                               std::vector<shape_t> shape, std::vector<shape_t> steps,
                               std::vector<int8_t> data) {
  for (shape_t b = 0; b < src_shape[0]; b++) {
    for (shape_t c = 0; c < src_shape[1]; c++) {
      for (shape_t i = 0; i < src_shape[2]; i++) {
        for (shape_t j = 0; j < src_shape[3]; j++) {
          for (shape_t block_b = 0; block_b < src_shape[4]; block_b++) {
            for (shape_t block_c = 0; block_c < src_shape[5]; block_c++) {
              data[dot_shape({b, c, i + pad_top, j + pad_left, block_b, block_c}, steps)] =
                  src_addr[dot_shape({b, c, i, j, block_b, block_c}, src_steps)];
            }
          }
        }
      }
    }
  }
  return data;
}

In [6]:
int trans_fixed_point_multiply_uint8(int8_t* shift, int32_t* pos_rounding_value)
{
	*shift = -*shift;		
	*shift = 8+*shift;
	//assert(*shift >= 0);
	*pos_rounding_value = (*shift >= 1) ? (1<< (*shift - 1)):0;	
	return 0;
}

In [ ]:
int8_t shift = 1;


In [7]:
int trans_fixed_point_multiply_int8(int8_t* shift, int32_t* pos_rounding_value)
{
	*shift = -*shift;		
	*shift =  7 + *shift;
	*pos_rounding_value = (*shift >= 1) ? (1<< (*shift - 1)):0;
	return 0;
}

In [8]:
int trans_fixed_point_multiply_shift(int32_t multiply_shift, uint8_t* multiply, int8_t* shift, int32_t* pos_rounding_value, int ms_nums=1)
{
	*shift = (multiply_shift & 0xff);
  *multiply = (multiply_shift >> 8) & 0xff;
	if (*multiply == 1 && *shift == 0){
    *pos_rounding_value = 0;
	} 
  else {
    //逐通道量化时硬件目前只支持multiply数据类型为int8, 逐tensor时可以支持uint8
    if (ms_nums > 1) {
      trans_fixed_point_multiply_int8(shift, pos_rounding_value);
    }
    else {
      trans_fixed_point_multiply_uint8(shift, pos_rounding_value);
    }
    
  }
  return 0;
}